# **Análise de Dados - Shoply**

## 1. ENTENDIMENTO DOS DADOS

### 1.1 Configuração inicial

In [1]:
# Bibliotecas principais
import pandas as pd              # Manipulação de dados
import numpy as np               # Operações numéricas
import matplotlib.pyplot as plt  # Visualizações básicas
import seaborn as sns            # Visualizações estatísticas
from google.colab import drive   # Upload de arquivos do drive
from datetime import datetime
from datetime import timedelta
from pathlib import Path

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import lightgbm as lgb
import shap

In [2]:
# Configurações de display
pd.set_option("display.max_columns", None)   # Mostrar todas as colunas
pd.set_option("display.float_format", "{:,.2f}".format)  # Format numérico
sns.set(style="whitegrid", palette="viridis")  # Estilo padrão de gráficos

### 1.2 Carregamento dos dados

In [4]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Caminho da pasta onde os arquivos estão
DATA_PATH = "/content/drive/MyDrive/Shoply - Desafio de dados/"

In [6]:
# Carregar cada base separadamente
orders_raw = pd.read_csv(DATA_PATH + "orders_dirty_final.csv")

### 1.3 Exploração Inicial de Dados

In [7]:
# Avaliar tamanho das bases
orders_raw.shape

(501500, 13)

In [8]:
# Avaliar head da base
orders_raw.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,NaN,boleto,PR,2018-01-27,2018-01-26
1,2,2,2018-01-28,delivered,82.06,0.00,2,acessorios,NaN,credit_card,RJ,2018-01-30,2018-01-30
2,3,3,2018-01-17,delivered,501.8,18.16,1,eletro,NaN,boleto,CE,2018-01-27,2018-01-27
3,4,4,2018-01-02,delivered,85.76,-3.30,2,acessorios,NaN,credit_card,SP,2018-01-04,2018-01-04
4,5,5,2018-01-29,delivered,43.77,0.00,3,acessorios,NaN,credit_card,SP,2018-01-31,2018-01-31


In [9]:
orders_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501500 entries, 0 to 501499
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   order_id                 501500 non-null  int64  
 1   customer_id              501500 non-null  int64  
 2   order_date               501500 non-null  object 
 3   order_status             501500 non-null  object 
 4   order_value              501500 non-null  object 
 5   discount_value           501498 non-null  float64
 6   sku_count                501303 non-null  object 
 7   order_category           501500 non-null  object 
 8   campaign_id              15408 non-null   object 
 9   payment_method           500492 non-null  object 
 10  delivery_state           501500 non-null  object 
 11  delivered_at             469122 non-null  object 
 12  estimated_delivery_date  501440 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 49.7+ MB


## 2. TRATAMENTO DOS DADOS

In [10]:
orders = orders_raw.copy()

#### Orders

Tipagem de dados

In [11]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501500 entries, 0 to 501499
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   order_id                 501500 non-null  int64  
 1   customer_id              501500 non-null  int64  
 2   order_date               501500 non-null  object 
 3   order_status             501500 non-null  object 
 4   order_value              501500 non-null  object 
 5   discount_value           501498 non-null  float64
 6   sku_count                501303 non-null  object 
 7   order_category           501500 non-null  object 
 8   campaign_id              15408 non-null   object 
 9   payment_method           500492 non-null  object 
 10  delivery_state           501500 non-null  object 
 11  delivered_at             469122 non-null  object 
 12  estimated_delivery_date  501440 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 49.7+ MB


In [12]:
# Garantir que IDs sejam tratados como strings e não números
orders["order_id"] = orders["order_id"].astype(str)
orders["customer_id"] = orders["customer_id"].astype(str)

# Tipagens corretas
orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")
orders["order_value"]= pd.to_numeric(orders["order_value"], errors="coerce")
orders["discount_value"]= pd.to_numeric(orders["discount_value"], errors="coerce")
orders["sku_count"]  = pd.to_numeric(orders["sku_count"], errors="coerce")
orders["delivered_at"] = pd.to_datetime(orders["delivered_at"], errors="coerce")
orders["estimated_delivery_date"] = pd.to_datetime(orders["estimated_delivery_date"], errors="coerce")

Duplicatas

In [13]:
# Contar quantas duplicatas existem
dup_mask = orders["order_id"].duplicated(keep=False)
print("Qtd de linhas duplicadas em order_id:", dup_mask.sum())

Qtd de linhas duplicadas em order_id: 3000


In [14]:
# Ver primeira amostra das duplicatas
dup_sample = orders.loc[dup_mask].sort_values("order_id").head(6)
print("\nAmostra das duplicatas:\n", dup_sample)


Amostra das duplicatas:
        order_id customer_id order_date order_status  order_value  \
100082   100083        1840 2020-12-08    delivered        45.35   
501112   100083        1840 2020-12-08    delivered        45.35   
500676   100390       66254 2020-12-22    delivered       518.77   
100389   100390       66254 2020-12-22    delivered       518.77   
501306   100960       66571 2021-01-04    delivered        60.60   
100959   100960       66571 2021-01-04    delivered        60.60   

        discount_value  sku_count order_category campaign_id payment_method  \
100082            7.37       3.00     acessorios         NaN    credit_card   
501112            7.37       3.00     acessorios         NaN    credit_card   
500676          100.78       3.00           casa         NaN    credit_card   
100389          100.78       3.00           casa         NaN    credit_card   
501306            5.12       2.00     acessorios         NaN    credit_card   
100959            5.12 

In [15]:
# remove duplicata de ID mantendo a 1ª ocorrência
orders = orders[~orders["order_id"].duplicated(keep="first")].copy()

Arrumar valores numéricos

In [16]:
orders.describe()

,order_date,order_value,discount_value,sku_count,delivered_at,estimated_delivery_date
count,500000,"498,999.00","499,998.00","499,407.00",467209,499940
mean,2022-11-03 10:36:52.300799744,267.28,17.44,2.08,2022-11-02 01:56:20.844974848,2022-11-07 18:32:22.850742272
min,2018-01-01 00:00:00,-10.00,-268.88,-1.00,2018-01-01 00:00:00,2018-01-03 00:00:00
25%,2021-06-21 00:00:00,66.39,4.36,1.00,2021-06-14 00:00:00,2021-06-26 00:00:00
50%,2023-02-06 00:00:00,104.32,9.13,2.00,2023-02-04 00:00:00,2023-02-11 00:00:00
75%,2024-06-17 00:00:00,208.83,18.67,3.00,2024-06-18 00:00:00,2024-06-21 00:00:00
max,2026-04-26 00:00:00,"99,317.00","2,105.90",4.00,2025-12-31 00:00:00,2026-01-02 00:00:00
std,NaN,"2,171.34",30.56,0.86,NaN,NaN


In [17]:
# order_value
neg_mask = orders["order_value"] < 0
neg_rate = neg_mask.mean()
print(f"Taxa de order_value < 0: {neg_rate:.2%}")

Taxa de order_value < 0: 0.26%


In [18]:
# Se < 5%, remover
removed_neg = int(neg_mask.sum())
orders = orders[~neg_mask].copy()

In [19]:
# Definir ideal pra desconto
mask_neg   = orders["discount_value"] < 0

n_total = len(orders)
n_neg   = int(mask_neg.sum())
print(f"Registros totais: {n_total}")
print(f"Inválidos (<0): {n_neg} ({n_neg/n_total:.2%})")

Registros totais: 498723
Inválidos (<0): 478 (0.10%)


In [20]:
# Correção
orders.loc[orders["discount_value"] < 0, "discount_value"] = 0

In [21]:
# sku_count inválido → regra conservadora: mínimo 1, inteiro
# tudo que não for número vira NaN já na tipagem; agora imputamos 1
orders.loc[orders["sku_count"].isna() | (orders["sku_count"] < 1), "sku_count"] = 1

# força inteiro
orders["sku_count"] = orders["sku_count"].round().astype(int)

Lógica delivery

In [22]:
# delivered_at antes da compra
mask_before_order = orders["delivered_at"].notna() & (orders["delivered_at"] < orders["order_date"])
# delivered_at preenchido quando não deveria
mask_shouldnt_have = orders["delivered_at"].notna() & orders["order_status"].isin(["cancelled","refunded","processing"])
# delivered_at vazio quando devia tá preenchido
mask_missing_but_should = orders["delivered_at"].isna() & orders["order_status"].isin(["delivered","returned"])
# estimated_delivery antes da compra
mask_bad_est = (orders["estimated_delivery_date"].notna() & (orders["estimated_delivery_date"] < orders["order_date"]))

print("Resumo inicial de erros detectados:\n")
print("Delivered_at antes da compra:", mask_before_order.sum())
print("Delivered_at preenchido quando não deveria:", mask_shouldnt_have.sum())
print("Delivered_at vazio quando devia tá preenchido:", mask_missing_but_should.sum())
print("Estimated_delivery antes da compra:", mask_bad_est.sum())

Resumo inicial de erros detectados:

Delivered_at antes da compra: 2629
Delivered_at preenchido quando não deveria: 985
Delivered_at vazio quando devia tá preenchido: 936
Estimated_delivery antes da compra: 2232


In [23]:
# 1) delivered_at antes de order_date → limpar
orders.loc[mask_before_order, "delivered_at"] = pd.NaT

In [24]:
# 2) delivered_at presente em status sem entrega → limpar
orders.loc[mask_shouldnt_have, "delivered_at"] = pd.NaT

In [25]:
# 3) Arrumar tempo estimado
# SLA simples por estado
sla_map = {
    "AC": 8,"AL": 8,"AM": 8,"AP": 8,"BA": 8,"CE": 8,"PA": 8,"PB": 8,"PE": 8,"PI": 8,"MA": 8,"RN": 8,"RO": 8,"RR": 8,"SE": 8,"TO": 8,
    "DF": 5,"GO": 5,"MT": 5,"MS": 5,"PR": 5,"SC": 5,"RS": 5,
    "SP": 3,"RJ": 3,"MG": 3,"ES": 3
}

orders["sla_days"] = orders["delivery_state"].map(sla_map).fillna(5)

orders.loc[mask_bad_est, "estimated_delivery_date"] = (
    orders.loc[mask_bad_est, "order_date"] +
    pd.to_timedelta(orders.loc[mask_bad_est, "sla_days"], unit="D")
)

In [26]:
# 4) delivered_at ausente mas deveria ter → imputar
#     regra: usar estimated_delivery_date; se NaT, usar order_date

mask_missing_but_should = orders["delivered_at"].isna() & orders["order_status"].isin(["delivered","returned"])
fill_est = orders.loc[mask_missing_but_should, "estimated_delivery_date"]
fill_ord = orders.loc[mask_missing_but_should, "order_date"]

orders.loc[mask_missing_but_should, "delivered_at"] = fill_est.fillna(fill_ord)

Normalização de campos categóricos

In [27]:
orders["payment_method"].unique()

array(['boleto', 'credit_card', 'paypal', '-', ' credit_card ', 'pixx',
       'credit_card  ', ' CREDIT_CARD  ', nan, ' CREDIT_CARD ',
       'paypal   ', ' credit_card', ' credit_card  ', 'CREDIT_CARD ',
       ' boleto  ', 'boleto_bancario', 'CREDITCARD', 'credit_card ',
       '(blank)', ' CREDIT_CARD', 'CREDIT_CARD', ' paypal ', ' BOLETO  ',
       'CREDIT_CARD  ', 'boleto ', 'BOLETO  ', ' boleto ', 'boleto  ',
       'BOLETO ', 'paypal  ', ' BOLETO', ' PAYPAL ', 'BOLETO', ' boleto',
       ' paypal', 'PAYPAL  ', 'pix', ' pix ', ' pix  ', 'PIX', 'pix  ',
       'PIX  ', ' pix', ' PAYPAL  ', 'PIX ', ' PIX', 'pix ', ' PIX  ',
       ' PIX ', ' BOLETO ', 'paypal ', 'PAYPAL', ' PIXX  ',
       ' PAYPAL     ', ' paypal  ', ' ', ' PAYPAL', 'creditcard  ',
       'NULL  ', 'PAYPAL '], dtype=object)

In [28]:
# Normalização discreta de campos categóricos para reduzir ruído inadvertido
if "payment_method" in orders.columns:
    orders["payment_method"] = orders["payment_method"].str.lower().str.strip()
if "delivery_state" in orders.columns:
    orders["delivery_state"] = orders["delivery_state"].str.upper().str.strip()
if "order_status" in orders.columns:
    orders["order_status"] = orders["order_status"].str.lower().str.strip()
if "order_category" in orders.columns:
    orders["order_category"] = orders["order_category"].str.lower().str.strip()

Nulos

In [29]:
# Nulos críticos
crit_cols_orders = ["order_id","customer_id","order_date","order_value"]
print(orders[crit_cols_orders].isna().sum())

order_id          0
customer_id       0
order_date        0
order_value    1001
dtype: int64


In [30]:
# Aplicar dropna
orders = orders.dropna(subset=["order_value"]).copy()

Arrumar payment methods

In [31]:
orders['payment_method'].unique()

array(['boleto', 'credit_card', 'paypal', '-', 'pixx', nan,
       'boleto_bancario', 'creditcard', '(blank)', 'pix', '', 'null'],
      dtype=object)

In [32]:
orders["payment_method"] = (
    orders["payment_method"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"": None, "nan": None, "null": None, "-": None, "(blank)": None})
    .replace({
        "creditcard": "credit_card",
        "credit_card": "credit_card",
        "pixx": "pix",
        "pix": "pix",
        "boleto_bancario": "boleto",
        "boleto": "boleto",
        "paypal": "paypal",
    })
)


Arrumar state

In [33]:
orders['delivery_state'].unique()

array(['PR', 'RJ', 'CE', 'SP', 'MG', 'AC', 'SC', 'RS', 'BA', 'PE', 'MA',
       'PB', 'MT', 'ES', 'RN', '??', 'PA', 'GO', 'RO', 'RR', 'TO', 'DF',
       'SE', 'AM', 'MS', 'AL', 'PI', 'AP', 'XX', 'BR'], dtype=object)

In [34]:
orders["delivery_state"] = (
    orders["delivery_state"]
    .astype(str)
    .str.strip()
    .replace({
        "??": "XX"
    })
)

Arrumar datas erradas

In [35]:
orders[orders["order_date"] > "2025-11-30"]

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days
3080,3081,2570,2026-01-02,delivered,46.75,1.58,2,acessorios,NaN,credit_card,RJ,2026-01-05,2026-01-05,3.00
3202,3203,2663,2026-01-01,delivered,144.29,10.20,2,acessorios,NaN,credit_card,PR,2026-01-06,2026-01-06,5.00
3759,3760,3052,2026-01-02,delivered,200.06,11.88,1,acessorios,NaN,credit_card,PA,2026-01-10,2026-01-10,8.00
3896,3897,776,2026-01-03,delivered,87.82,0.00,3,acessorios,NaN,credit_card,PR,2026-01-08,2026-01-08,5.00
9169,9170,5501,2026-01-01,delivered,92.59,0.85,3,acessorios,NaN,credit_card,MG,2026-01-04,2026-01-04,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497878,497879,107757,2025-12-22,delivered,45.87,13.54,3,acessorios,NaN,pix,BA,2025-12-30,2025-12-30,8.00
497935,497936,277835,2026-03-31,delivered,173.53,32.43,3,casa,NaN,credit_card,RJ,2026-04-03,2026-04-03,3.00
498621,498622,242735,2026-03-14,delivered,464.16,77.79,2,eletro,NaN,pix,MG,2026-03-17,2026-03-17,3.00
498704,498705,278276,2026-01-01,delivered,58.18,18.26,3,beleza,NaN,boleto,AL,2026-01-09,2026-01-09,8.00


In [36]:
orders = orders[orders["order_date"] <= "2025-11-30"]

Arrumar campanhas

In [37]:
orders['campaign_id'].nunique()

1079

In [38]:
orders["campaign_id"] = (
    orders["campaign_id"]
    .astype(str)
    .str.strip()
    .replace({"": None, "nan": None, "null": None, "-": None, "(blank)": None, "N/A": None})
)

## 3. CRIAÇÃO DE FEATURES

In [39]:
orders.to_excel("orders_clean.xlsx")

### Features de pedido

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00
1,2,2,2018-01-28,delivered,82.06,0.00,2,acessorios,None,credit_card,RJ,2018-01-30,2018-01-30,3.00
2,3,3,2018-01-17,delivered,501.80,18.16,1,eletro,None,boleto,CE,2018-01-27,2018-01-27,8.00
3,4,4,2018-01-02,delivered,85.76,0.00,2,acessorios,None,credit_card,SP,2018-01-04,2018-01-04,3.00
4,5,5,2018-01-29,delivered,43.77,0.00,3,acessorios,None,credit_card,SP,2018-01-31,2018-01-31,3.00


In [ ]:
# primeira compra do cliente
orders["is_new_customer"] = (
    orders.groupby("customer_id")["order_date"].transform("min") == orders["order_date"]
).astype(int)

In [ ]:
# Valor bruto e métricas de entrega
orders["total_order_value"] = orders["order_value"] + orders["discount_value"]
orders["pct_discount"] = orders["discount_value"]/orders["total_order_value"]
orders["days_to_delivery"] = (orders["delivered_at"] - orders["order_date"]).dt.days
orders["delivery_delay_days"] = (orders["delivered_at"] - orders["estimated_delivery_date"]).dt.days
orders["is_late_delivery"] = (orders["delivery_delay_days"] > 0).astype("Int8")

In [ ]:
# Particionamento temporal
orders["order_year"]  = orders["order_date"].dt.year
orders["order_month"] = orders["order_date"].dt.month
orders["order_month_year"] = orders["order_date"].dt.to_period("M")
orders["order_week"]  = orders["order_date"].dt.isocalendar().week.astype("Int64")

In [ ]:
# Sazonalidade
orders["holiday_tag"] = ""
orders.loc[orders["order_date"].dt.month == 11, "holiday_tag"] = "black_friday"
orders.loc[orders["order_date"].dt.month == 12, "holiday_tag"] = "christmas"
orders.loc[orders["order_date"].dt.month == 5,  "holiday_tag"] = "mothers_day"
orders["holiday_tag"] = orders["holiday_tag"].replace("", "normal")

In [ ]:
# Calcular quartis do ticket líquido
q1, q2, q3 = orders["order_value"].quantile([0.25, 0.50, 0.75])

print(f"Quartis do net_order_value:\nQ1={q1:.2f} | Q2={q2:.2f} | Q3={q3:.2f}")

Quartis do net_order_value:
Q1=66.64 | Q2=104.56 | Q3=209.37


In [ ]:
# Classificar ticket em 4 faixas
orders["basket_size_flag"] = pd.cut(
    orders["order_value"],
    bins=[-1, q1, q2, q3, orders["order_value"].max()],
    labels=["Q1_low","Q2_mid_low","Q3_mid_high","Q4_high"]
)

# High-ticket definido como Q4
orders["high_ticket_flag"] = (orders["order_value"] >= q3).astype("Int8")

In [ ]:
# Histórico incremental por cliente
orders = orders.sort_values(["customer_id","order_date","order_id"])
orders["customer_lifetime_orders"] = orders.groupby("customer_id").cumcount() + 1 # Número acumulado de pedidos por cliente (1ª, 2ª, 3ª compra etc.)
orders["customer_lifetime_gmv"] = orders.groupby("customer_id")["order_value"].cumsum() # GMV acumulado até o pedido atual

In [ ]:
# Ticket médio até o pedido atual
orders["customer_avg_ticket_to_date"] = (
    orders["customer_lifetime_gmv"] / orders["customer_lifetime_orders"]
)

In [ ]:
# Dias desde a última compra antes da compra atual
orders["prev_order_date"] = orders.groupby("customer_id")["order_date"].shift(1) # Data do pedido anterior do mesmo cliente
orders["days_since_last_order_before_purchase"] = (
    orders["order_date"] - orders["prev_order_date"]
).dt.days

In [ ]:
# Region (via UF) — nível pedido
uf_to_region = {
    "SP":"Sudeste","RJ":"Sudeste","MG":"Sudeste","ES":"Sudeste",
    "PR":"Sul","SC":"Sul","RS":"Sul",
    "DF":"Centro-Oeste","GO":"Centro-Oeste","MT":"Centro-Oeste","MS":"Centro-Oeste",
    "BA":"Nordeste","PE":"Nordeste","CE":"Nordeste","RN":"Nordeste","PB":"Nordeste","MA":"Nordeste","AL":"Nordeste","SE":"Nordeste","PI":"Nordeste",
    "PA":"Norte","AM":"Norte","RO":"Norte","RR":"Norte","AC":"Norte","AP":"Norte","TO":"Norte"
}
orders["region"] = orders["delivery_state"].map(uf_to_region).fillna("Desconhecida")


In [ ]:
# Identificar "novos" vs "recorrentes"

# Um cliente é "novo" se essa order é a primeira dele na série
orders = orders.sort_values(["customer_id", "order_date"])
orders["is_new_customer_period"] = orders["prev_order_date"].isna()


In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
orders.to_excel("orders_features.xlsx")

### Features de diagnóstico

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Base de métricas gerais por mês pra facilitar diagnóstico

monthly = (
    orders.groupby("order_month_year")
    .agg(
        total_gmv=("order_value", "sum"),
        total_orders=("order_id", "nunique"),
        active_customers=("customer_id", "nunique"),
    )
)

monthly["aov"] = monthly["total_gmv"] / monthly["total_orders"]
monthly["orders_per_customer"] = monthly["total_orders"] / monthly["active_customers"]

In [ ]:
# Base auxiliar de infos pdiferenciando novos clientes de antigos
seg = (
    orders.groupby(["order_month_year", "is_new_customer_period"])
    .agg(
        gmv=("order_value", "sum"),
        orders=("order_id", "nunique"),
        customers=("customer_id", "nunique"),
    )
    .reset_index()
)

# Pivot para ficar legível
seg = seg.pivot(
    index="order_month_year",
    columns="is_new_customer_period",
    values=["gmv", "orders", "customers"]
)

seg.columns = [
    f"{metric}_{'new' if flag else 'returning'}"
    for metric, flag in seg.columns]

In [ ]:
seg.head()

,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,
2018-01,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Unir tudo

# garantir formatos iguais para o join
monthly.index = monthly.index.astype(str)
seg.index = seg.index.astype(str)

diagnostic = monthly.join(seg, how="left")

In [ ]:
diagnostic.head()

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Métricas derivadas

diagnostic["gmv_share_new"] = diagnostic["gmv_new"] / diagnostic["total_gmv"]

diagnostic["share_new"] = diagnostic["orders_new"] / diagnostic["total_orders"]

diagnostic["aov_new"] = diagnostic["gmv_new"] / diagnostic["orders_new"]
diagnostic["aov_returning"] = diagnostic["gmv_returning"] / diagnostic["orders_returning"] #Average Order Value.

diagnostic["orders_per_customer_new"] = (
    diagnostic["orders_new"] / diagnostic["customers_new"]
)
diagnostic["orders_per_customer_returning"] = (
    diagnostic["orders_returning"] / diagnostic["customers_returning"]
)

diagnostic.head(6)

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new,gmv_share_new,share_new,aov_new,aov_returning,orders_per_customer_new,orders_per_customer_returning
order_month_year,,,,,,,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00",1.00,1.00,237.74,NaN,1.00,NaN
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00",0.79,0.70,213.35,130.23,1.00,1.25
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00",0.79,0.70,203.77,130.53,1.00,1.13
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00",0.76,0.70,183.63,135.54,1.00,1.10
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00",0.73,0.70,246.28,218.11,1.00,1.05
2018-06,"350,946.26",1868,1844,187.87,1.01,"110,661.96","240,284.30",558.00,"1,310.00",534.00,"1,310.00",0.68,0.70,183.42,198.32,1.00,1.04


In [ ]:
diagnostic.columns

Index(['total_gmv', 'total_orders', 'active_customers', 'aov',
       'orders_per_customer', 'gmv_returning', 'gmv_new', 'orders_returning',
       'orders_new', 'customers_returning', 'customers_new', 'gmv_share_new',
       'share_new', 'aov_new', 'aov_returning', 'orders_per_customer_new',
       'orders_per_customer_returning'],
      dtype='object')

### Features de clientes

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Construção da base customers a partir de orders
customers = (
    orders[["customer_id"]]
    .dropna()
    .drop_duplicates()
    .sort_values("customer_id")
    .reset_index(drop=True)
)

In [ ]:
customers.head()

,customer_id
0,1
1,10
2,100
3,1000
4,10000


In [ ]:
# Período de relacionamento

tmp = (
    orders.groupby("customer_id")["order_date"]
    .agg(first_order_date="min", last_order_date="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

today = pd.Timestamp.today().normalize()  # data de hoje sem horário

customers["total_days_as_customer"] = (today - customers["first_order_date"]).dt.days

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
# Volume e valor

tmp = (
    orders.groupby("customer_id")
    .agg(
        total_orders=("order_id","count"),
        total_gmv=("order_value","sum"),
        avg_order_value=("order_value","mean"),
        median_order_value=("order_value","median"),
        total_discount=("discount_value","sum"),
        avg_pct_discount=("pct_discount","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mix e perfil de compra

tmp = (
    orders.groupby("customer_id")
    .agg(
        unique_categories=("order_category","nunique"),
        avg_sku_per_order=("sku_count","mean"),
        pct_high_ticket=("high_ticket_flag","mean"),  # proporção Q4
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Logística e experiência

tmp = (
    orders.groupby("customer_id")
    .agg(
        avg_days_to_delivery=("days_to_delivery","mean"),
        pct_deliveries_late=("is_late_delivery","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Região mais frequente

tmp = (
    orders.groupby("customer_id")["region"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_frequent_region")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Recompra e relacionamento

tmp = (
    orders.groupby("customer_id")["days_since_last_order_before_purchase"]
    .agg(avg_days_between_orders="mean", max_days_between_orders="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Método de pagamento mais usado

tmp = (
    orders.groupby("customer_id")["payment_method"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_used_payment")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mais algumas temporais

latest_date = orders["order_date"].max()
customers["days_since_last_order"] = (today - customers["last_order_date"]).dt.days
customers["is_active_90d"] = (customers["days_since_last_order"] <= 90).astype("Int8")

In [ ]:
# Coorte de entrada
customers["first_order_date"] = pd.to_datetime(customers["first_order_date"], errors="coerce")
customers["cohort_month"] = customers["first_order_date"].dt.to_period("M").astype(str)

In [ ]:
customers.columns

Index(['customer_id', 'first_order_date', 'last_order_date',
       'total_days_as_customer', 'total_orders', 'total_gmv',
       'avg_order_value', 'median_order_value', 'total_discount',
       'avg_pct_discount', 'unique_categories', 'avg_sku_per_order',
       'pct_high_ticket', 'avg_days_to_delivery', 'pct_deliveries_late',
       'most_frequent_region', 'avg_days_between_orders',
       'max_days_between_orders', 'most_used_payment', 'days_since_last_order',
       'is_active_90d', 'cohort_month'],
      dtype='object')

In [ ]:
customers.to_excel("customers_features.xlsx")

## 3. CRIAÇÃO DE FEATURES

In [ ]:
orders.to_excel("orders_clean.xlsx")

### Features de pedido

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00
1,2,2,2018-01-28,delivered,82.06,0.00,2,acessorios,None,credit_card,RJ,2018-01-30,2018-01-30,3.00
2,3,3,2018-01-17,delivered,501.80,18.16,1,eletro,None,boleto,CE,2018-01-27,2018-01-27,8.00
3,4,4,2018-01-02,delivered,85.76,0.00,2,acessorios,None,credit_card,SP,2018-01-04,2018-01-04,3.00
4,5,5,2018-01-29,delivered,43.77,0.00,3,acessorios,None,credit_card,SP,2018-01-31,2018-01-31,3.00


In [ ]:
# primeira compra do cliente
orders["is_new_customer"] = (
    orders.groupby("customer_id")["order_date"].transform("min") == orders["order_date"]
).astype(int)

In [ ]:
# Valor bruto e métricas de entrega
orders["total_order_value"] = orders["order_value"] + orders["discount_value"]
orders["pct_discount"] = orders["discount_value"]/orders["total_order_value"]
orders["days_to_delivery"] = (orders["delivered_at"] - orders["order_date"]).dt.days
orders["delivery_delay_days"] = (orders["delivered_at"] - orders["estimated_delivery_date"]).dt.days
orders["is_late_delivery"] = (orders["delivery_delay_days"] > 0).astype("Int8")

In [ ]:
# Particionamento temporal
orders["order_year"]  = orders["order_date"].dt.year
orders["order_month"] = orders["order_date"].dt.month
orders["order_month_year"] = orders["order_date"].dt.to_period("M")
orders["order_week"]  = orders["order_date"].dt.isocalendar().week.astype("Int64")

In [ ]:
# Sazonalidade
orders["holiday_tag"] = ""
orders.loc[orders["order_date"].dt.month == 11, "holiday_tag"] = "black_friday"
orders.loc[orders["order_date"].dt.month == 12, "holiday_tag"] = "christmas"
orders.loc[orders["order_date"].dt.month == 5,  "holiday_tag"] = "mothers_day"
orders["holiday_tag"] = orders["holiday_tag"].replace("", "normal")

In [ ]:
# Calcular quartis do ticket líquido
q1, q2, q3 = orders["order_value"].quantile([0.25, 0.50, 0.75])

print(f"Quartis do net_order_value:\nQ1={q1:.2f} | Q2={q2:.2f} | Q3={q3:.2f}")

Quartis do net_order_value:
Q1=66.64 | Q2=104.56 | Q3=209.37


In [ ]:
# Classificar ticket em 4 faixas
orders["basket_size_flag"] = pd.cut(
    orders["order_value"],
    bins=[-1, q1, q2, q3, orders["order_value"].max()],
    labels=["Q1_low","Q2_mid_low","Q3_mid_high","Q4_high"]
)

# High-ticket definido como Q4
orders["high_ticket_flag"] = (orders["order_value"] >= q3).astype("Int8")

In [ ]:
# Histórico incremental por cliente
orders = orders.sort_values(["customer_id","order_date","order_id"])
orders["customer_lifetime_orders"] = orders.groupby("customer_id").cumcount() + 1 # Número acumulado de pedidos por cliente (1ª, 2ª, 3ª compra etc.)
orders["customer_lifetime_gmv"] = orders.groupby("customer_id")["order_value"].cumsum() # GMV acumulado até o pedido atual

In [ ]:
# Ticket médio até o pedido atual
orders["customer_avg_ticket_to_date"] = (
    orders["customer_lifetime_gmv"] / orders["customer_lifetime_orders"]
)

In [ ]:
# Dias desde a última compra antes da compra atual
orders["prev_order_date"] = orders.groupby("customer_id")["order_date"].shift(1) # Data do pedido anterior do mesmo cliente
orders["days_since_last_order_before_purchase"] = (
    orders["order_date"] - orders["prev_order_date"]
).dt.days

In [ ]:
# Region (via UF) — nível pedido
uf_to_region = {
    "SP":"Sudeste","RJ":"Sudeste","MG":"Sudeste","ES":"Sudeste",
    "PR":"Sul","SC":"Sul","RS":"Sul",
    "DF":"Centro-Oeste","GO":"Centro-Oeste","MT":"Centro-Oeste","MS":"Centro-Oeste",
    "BA":"Nordeste","PE":"Nordeste","CE":"Nordeste","RN":"Nordeste","PB":"Nordeste","MA":"Nordeste","AL":"Nordeste","SE":"Nordeste","PI":"Nordeste",
    "PA":"Norte","AM":"Norte","RO":"Norte","RR":"Norte","AC":"Norte","AP":"Norte","TO":"Norte"
}
orders["region"] = orders["delivery_state"].map(uf_to_region).fillna("Desconhecida")


In [ ]:
# Identificar "novos" vs "recorrentes"

# Um cliente é "novo" se essa order é a primeira dele na série
orders = orders.sort_values(["customer_id", "order_date"])
orders["is_new_customer_period"] = orders["prev_order_date"].isna()


In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
orders.to_excel("orders_features.xlsx")

### Features de diagnóstico

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Base de métricas gerais por mês pra facilitar diagnóstico

monthly = (
    orders.groupby("order_month_year")
    .agg(
        total_gmv=("order_value", "sum"),
        total_orders=("order_id", "nunique"),
        active_customers=("customer_id", "nunique"),
    )
)

monthly["aov"] = monthly["total_gmv"] / monthly["total_orders"]
monthly["orders_per_customer"] = monthly["total_orders"] / monthly["active_customers"]

In [ ]:
# Base auxiliar de infos pdiferenciando novos clientes de antigos
seg = (
    orders.groupby(["order_month_year", "is_new_customer_period"])
    .agg(
        gmv=("order_value", "sum"),
        orders=("order_id", "nunique"),
        customers=("customer_id", "nunique"),
    )
    .reset_index()
)

# Pivot para ficar legível
seg = seg.pivot(
    index="order_month_year",
    columns="is_new_customer_period",
    values=["gmv", "orders", "customers"]
)

seg.columns = [
    f"{metric}_{'new' if flag else 'returning'}"
    for metric, flag in seg.columns]

In [ ]:
seg.head()

,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,
2018-01,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Unir tudo

# garantir formatos iguais para o join
monthly.index = monthly.index.astype(str)
seg.index = seg.index.astype(str)

diagnostic = monthly.join(seg, how="left")

In [ ]:
diagnostic.head()

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Métricas derivadas

diagnostic["gmv_share_new"] = diagnostic["gmv_new"] / diagnostic["total_gmv"]

diagnostic["share_new"] = diagnostic["orders_new"] / diagnostic["total_orders"]

diagnostic["aov_new"] = diagnostic["gmv_new"] / diagnostic["orders_new"]
diagnostic["aov_returning"] = diagnostic["gmv_returning"] / diagnostic["orders_returning"] #Average Order Value.

diagnostic["orders_per_customer_new"] = (
    diagnostic["orders_new"] / diagnostic["customers_new"]
)
diagnostic["orders_per_customer_returning"] = (
    diagnostic["orders_returning"] / diagnostic["customers_returning"]
)

diagnostic.head(6)

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new,gmv_share_new,share_new,aov_new,aov_returning,orders_per_customer_new,orders_per_customer_returning
order_month_year,,,,,,,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00",1.00,1.00,237.74,NaN,1.00,NaN
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00",0.79,0.70,213.35,130.23,1.00,1.25
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00",0.79,0.70,203.77,130.53,1.00,1.13
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00",0.76,0.70,183.63,135.54,1.00,1.10
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00",0.73,0.70,246.28,218.11,1.00,1.05
2018-06,"350,946.26",1868,1844,187.87,1.01,"110,661.96","240,284.30",558.00,"1,310.00",534.00,"1,310.00",0.68,0.70,183.42,198.32,1.00,1.04


In [ ]:
diagnostic.columns

Index(['total_gmv', 'total_orders', 'active_customers', 'aov',
       'orders_per_customer', 'gmv_returning', 'gmv_new', 'orders_returning',
       'orders_new', 'customers_returning', 'customers_new', 'gmv_share_new',
       'share_new', 'aov_new', 'aov_returning', 'orders_per_customer_new',
       'orders_per_customer_returning'],
      dtype='object')

### Features de clientes

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Construção da base customers a partir de orders
customers = (
    orders[["customer_id"]]
    .dropna()
    .drop_duplicates()
    .sort_values("customer_id")
    .reset_index(drop=True)
)

In [ ]:
customers.head()

,customer_id
0,1
1,10
2,100
3,1000
4,10000


In [ ]:
# Período de relacionamento

tmp = (
    orders.groupby("customer_id")["order_date"]
    .agg(first_order_date="min", last_order_date="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

today = pd.Timestamp.today().normalize()  # data de hoje sem horário

customers["total_days_as_customer"] = (today - customers["first_order_date"]).dt.days

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
# Volume e valor

tmp = (
    orders.groupby("customer_id")
    .agg(
        total_orders=("order_id","count"),
        total_gmv=("order_value","sum"),
        avg_order_value=("order_value","mean"),
        median_order_value=("order_value","median"),
        total_discount=("discount_value","sum"),
        avg_pct_discount=("pct_discount","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mix e perfil de compra

tmp = (
    orders.groupby("customer_id")
    .agg(
        unique_categories=("order_category","nunique"),
        avg_sku_per_order=("sku_count","mean"),
        pct_high_ticket=("high_ticket_flag","mean"),  # proporção Q4
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Logística e experiência

tmp = (
    orders.groupby("customer_id")
    .agg(
        avg_days_to_delivery=("days_to_delivery","mean"),
        pct_deliveries_late=("is_late_delivery","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Região mais frequente

tmp = (
    orders.groupby("customer_id")["region"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_frequent_region")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Recompra e relacionamento

tmp = (
    orders.groupby("customer_id")["days_since_last_order_before_purchase"]
    .agg(avg_days_between_orders="mean", max_days_between_orders="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Método de pagamento mais usado

tmp = (
    orders.groupby("customer_id")["payment_method"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_used_payment")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mais algumas temporais

latest_date = orders["order_date"].max()
customers["days_since_last_order"] = (today - customers["last_order_date"]).dt.days
customers["is_active_90d"] = (customers["days_since_last_order"] <= 90).astype("Int8")

In [ ]:
# Coorte de entrada
customers["first_order_date"] = pd.to_datetime(customers["first_order_date"], errors="coerce")
customers["cohort_month"] = customers["first_order_date"].dt.to_period("M").astype(str)

In [ ]:
customers.columns

Index(['customer_id', 'first_order_date', 'last_order_date',
       'total_days_as_customer', 'total_orders', 'total_gmv',
       'avg_order_value', 'median_order_value', 'total_discount',
       'avg_pct_discount', 'unique_categories', 'avg_sku_per_order',
       'pct_high_ticket', 'avg_days_to_delivery', 'pct_deliveries_late',
       'most_frequent_region', 'avg_days_between_orders',
       'max_days_between_orders', 'most_used_payment', 'days_since_last_order',
       'is_active_90d', 'cohort_month'],
      dtype='object')

In [ ]:
customers.to_excel("customers_features.xlsx")

## 3. CRIAÇÃO DE FEATURES

In [ ]:
orders.to_excel("orders_clean.xlsx")

### Features de pedido

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00
1,2,2,2018-01-28,delivered,82.06,0.00,2,acessorios,None,credit_card,RJ,2018-01-30,2018-01-30,3.00
2,3,3,2018-01-17,delivered,501.80,18.16,1,eletro,None,boleto,CE,2018-01-27,2018-01-27,8.00
3,4,4,2018-01-02,delivered,85.76,0.00,2,acessorios,None,credit_card,SP,2018-01-04,2018-01-04,3.00
4,5,5,2018-01-29,delivered,43.77,0.00,3,acessorios,None,credit_card,SP,2018-01-31,2018-01-31,3.00


In [ ]:
# primeira compra do cliente
orders["is_new_customer"] = (
    orders.groupby("customer_id")["order_date"].transform("min") == orders["order_date"]
).astype(int)

In [ ]:
# Valor bruto e métricas de entrega
orders["total_order_value"] = orders["order_value"] + orders["discount_value"]
orders["pct_discount"] = orders["discount_value"]/orders["total_order_value"]
orders["days_to_delivery"] = (orders["delivered_at"] - orders["order_date"]).dt.days
orders["delivery_delay_days"] = (orders["delivered_at"] - orders["estimated_delivery_date"]).dt.days
orders["is_late_delivery"] = (orders["delivery_delay_days"] > 0).astype("Int8")

In [ ]:
# Particionamento temporal
orders["order_year"]  = orders["order_date"].dt.year
orders["order_month"] = orders["order_date"].dt.month
orders["order_month_year"] = orders["order_date"].dt.to_period("M")
orders["order_week"]  = orders["order_date"].dt.isocalendar().week.astype("Int64")

In [ ]:
# Sazonalidade
orders["holiday_tag"] = ""
orders.loc[orders["order_date"].dt.month == 11, "holiday_tag"] = "black_friday"
orders.loc[orders["order_date"].dt.month == 12, "holiday_tag"] = "christmas"
orders.loc[orders["order_date"].dt.month == 5,  "holiday_tag"] = "mothers_day"
orders["holiday_tag"] = orders["holiday_tag"].replace("", "normal")

In [ ]:
# Calcular quartis do ticket líquido
q1, q2, q3 = orders["order_value"].quantile([0.25, 0.50, 0.75])

print(f"Quartis do net_order_value:\nQ1={q1:.2f} | Q2={q2:.2f} | Q3={q3:.2f}")

Quartis do net_order_value:
Q1=66.64 | Q2=104.56 | Q3=209.37


In [ ]:
# Classificar ticket em 4 faixas
orders["basket_size_flag"] = pd.cut(
    orders["order_value"],
    bins=[-1, q1, q2, q3, orders["order_value"].max()],
    labels=["Q1_low","Q2_mid_low","Q3_mid_high","Q4_high"]
)

# High-ticket definido como Q4
orders["high_ticket_flag"] = (orders["order_value"] >= q3).astype("Int8")

In [ ]:
# Histórico incremental por cliente
orders = orders.sort_values(["customer_id","order_date","order_id"])
orders["customer_lifetime_orders"] = orders.groupby("customer_id").cumcount() + 1 # Número acumulado de pedidos por cliente (1ª, 2ª, 3ª compra etc.)
orders["customer_lifetime_gmv"] = orders.groupby("customer_id")["order_value"].cumsum() # GMV acumulado até o pedido atual

In [ ]:
# Ticket médio até o pedido atual
orders["customer_avg_ticket_to_date"] = (
    orders["customer_lifetime_gmv"] / orders["customer_lifetime_orders"]
)

In [ ]:
# Dias desde a última compra antes da compra atual
orders["prev_order_date"] = orders.groupby("customer_id")["order_date"].shift(1) # Data do pedido anterior do mesmo cliente
orders["days_since_last_order_before_purchase"] = (
    orders["order_date"] - orders["prev_order_date"]
).dt.days

In [ ]:
# Region (via UF) — nível pedido
uf_to_region = {
    "SP":"Sudeste","RJ":"Sudeste","MG":"Sudeste","ES":"Sudeste",
    "PR":"Sul","SC":"Sul","RS":"Sul",
    "DF":"Centro-Oeste","GO":"Centro-Oeste","MT":"Centro-Oeste","MS":"Centro-Oeste",
    "BA":"Nordeste","PE":"Nordeste","CE":"Nordeste","RN":"Nordeste","PB":"Nordeste","MA":"Nordeste","AL":"Nordeste","SE":"Nordeste","PI":"Nordeste",
    "PA":"Norte","AM":"Norte","RO":"Norte","RR":"Norte","AC":"Norte","AP":"Norte","TO":"Norte"
}
orders["region"] = orders["delivery_state"].map(uf_to_region).fillna("Desconhecida")


In [ ]:
# Identificar "novos" vs "recorrentes"

# Um cliente é "novo" se essa order é a primeira dele na série
orders = orders.sort_values(["customer_id", "order_date"])
orders["is_new_customer_period"] = orders["prev_order_date"].isna()


In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
orders.to_excel("orders_features.xlsx")

### Features de diagnóstico

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Base de métricas gerais por mês pra facilitar diagnóstico

monthly = (
    orders.groupby("order_month_year")
    .agg(
        total_gmv=("order_value", "sum"),
        total_orders=("order_id", "nunique"),
        active_customers=("customer_id", "nunique"),
    )
)

monthly["aov"] = monthly["total_gmv"] / monthly["total_orders"]
monthly["orders_per_customer"] = monthly["total_orders"] / monthly["active_customers"]

In [ ]:
# Base auxiliar de infos pdiferenciando novos clientes de antigos
seg = (
    orders.groupby(["order_month_year", "is_new_customer_period"])
    .agg(
        gmv=("order_value", "sum"),
        orders=("order_id", "nunique"),
        customers=("customer_id", "nunique"),
    )
    .reset_index()
)

# Pivot para ficar legível
seg = seg.pivot(
    index="order_month_year",
    columns="is_new_customer_period",
    values=["gmv", "orders", "customers"]
)

seg.columns = [
    f"{metric}_{'new' if flag else 'returning'}"
    for metric, flag in seg.columns]

In [ ]:
seg.head()

,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,
2018-01,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Unir tudo

# garantir formatos iguais para o join
monthly.index = monthly.index.astype(str)
seg.index = seg.index.astype(str)

diagnostic = monthly.join(seg, how="left")

In [ ]:
diagnostic.head()

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new
order_month_year,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00"
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00"
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00"
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00"
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00"


In [ ]:
# Métricas derivadas

diagnostic["gmv_share_new"] = diagnostic["gmv_new"] / diagnostic["total_gmv"]

diagnostic["share_new"] = diagnostic["orders_new"] / diagnostic["total_orders"]

diagnostic["aov_new"] = diagnostic["gmv_new"] / diagnostic["orders_new"]
diagnostic["aov_returning"] = diagnostic["gmv_returning"] / diagnostic["orders_returning"] #Average Order Value.

diagnostic["orders_per_customer_new"] = (
    diagnostic["orders_new"] / diagnostic["customers_new"]
)
diagnostic["orders_per_customer_returning"] = (
    diagnostic["orders_returning"] / diagnostic["customers_returning"]
)

diagnostic.head(6)

,total_gmv,total_orders,active_customers,aov,orders_per_customer,gmv_returning,gmv_new,orders_returning,orders_new,customers_returning,customers_new,gmv_share_new,share_new,aov_new,aov_returning,orders_per_customer_new,orders_per_customer_returning
order_month_year,,,,,,,,,,,,,,,,,
2018-01,"330,690.26",1391,1391,237.74,1.00,NaN,"330,690.26",NaN,"1,391.00",NaN,"1,391.00",1.00,1.00,237.74,NaN,1.00,NaN
2018-02,"299,961.84",1591,1496,188.54,1.06,"61,860.04","238,101.80",475.00,"1,116.00",381.00,"1,116.00",0.79,0.70,213.35,130.23,1.00,1.25
2018-03,"313,461.72",1722,1662,182.03,1.04,"66,702.15","246,759.57",511.00,"1,211.00",452.00,"1,211.00",0.79,0.70,203.77,130.53,1.00,1.13
2018-04,"305,153.42",1804,1756,169.15,1.03,"73,597.07","231,556.35",543.00,"1,261.00",495.00,"1,261.00",0.76,0.70,183.63,135.54,1.00,1.10
2018-05,"452,670.43",1903,1875,237.87,1.01,"123,886.49","328,783.94",568.00,"1,335.00",540.00,"1,335.00",0.73,0.70,246.28,218.11,1.00,1.05
2018-06,"350,946.26",1868,1844,187.87,1.01,"110,661.96","240,284.30",558.00,"1,310.00",534.00,"1,310.00",0.68,0.70,183.42,198.32,1.00,1.04


In [ ]:
diagnostic.columns

Index(['total_gmv', 'total_orders', 'active_customers', 'aov',
       'orders_per_customer', 'gmv_returning', 'gmv_new', 'orders_returning',
       'orders_new', 'customers_returning', 'customers_new', 'gmv_share_new',
       'share_new', 'aov_new', 'aov_returning', 'orders_per_customer_new',
       'orders_per_customer_returning'],
      dtype='object')

### Features de clientes

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
orders.head()

,order_id,customer_id,order_date,order_status,order_value,discount_value,sku_count,order_category,campaign_id,payment_method,delivery_state,delivered_at,estimated_delivery_date,sla_days,is_new_customer,total_order_value,pct_discount,days_to_delivery,delivery_delay_days,is_late_delivery,order_year,order_month,order_month_year,order_week,holiday_tag,basket_size_flag,high_ticket_flag,customer_lifetime_orders,customer_lifetime_gmv,customer_avg_ticket_to_date,prev_order_date,days_since_last_order_before_purchase,region,is_new_customer_period
0,1,1,2018-01-21,delivered,44.26,1.78,4,acessorios,None,boleto,PR,2018-01-27,2018-01-26,5.00,1,46.04,0.04,6.00,1.00,1,2018,1,2018-01,3,normal,Q1_low,0,1,44.26,44.26,NaT,NaN,Sul,True
9,10,10,2018-01-14,delivered,83.36,2.59,2,acessorios,None,credit_card,AC,2018-01-25,2018-01-25,8.00,1,85.95,0.03,11.00,0.00,0,2018,1,2018-01,2,normal,Q2_mid_low,0,1,83.36,83.36,NaT,NaN,Norte,True
3172,3173,10,2018-03-30,delivered,106.04,5.07,2,acessorios,None,credit_card,AC,2018-04-14,2018-04-13,8.00,0,111.11,0.05,15.00,1.00,1,2018,3,2018-03,13,normal,Q3_mid_high,0,2,189.40,94.70,2018-01-14,75.00,Norte,False
99,100,100,2018-01-03,delivered,35.46,1.19,3,acessorios,None,credit_card,SP,2018-01-05,2018-01-05,3.00,1,36.65,0.03,2.00,0.00,0,2018,1,2018-01,1,normal,Q1_low,0,1,35.46,35.46,NaT,NaN,Sudeste,True
999,1000,1000,2018-01-26,delivered,65.73,4.37,2,acessorios,None,credit_card,RJ,2018-01-29,2018-01-29,3.00,1,70.10,0.06,3.00,0.00,0,2018,1,2018-01,4,normal,Q1_low,0,1,65.73,65.73,NaT,NaN,Sudeste,True


In [ ]:
# Construção da base customers a partir de orders
customers = (
    orders[["customer_id"]]
    .dropna()
    .drop_duplicates()
    .sort_values("customer_id")
    .reset_index(drop=True)
)

In [ ]:
customers.head()

,customer_id
0,1
1,10
2,100
3,1000
4,10000


In [ ]:
# Período de relacionamento

tmp = (
    orders.groupby("customer_id")["order_date"]
    .agg(first_order_date="min", last_order_date="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

today = pd.Timestamp.today().normalize()  # data de hoje sem horário

customers["total_days_as_customer"] = (today - customers["first_order_date"]).dt.days

In [ ]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'order_status', 'order_value',
       'discount_value', 'sku_count', 'order_category', 'campaign_id',
       'payment_method', 'delivery_state', 'delivered_at',
       'estimated_delivery_date', 'sla_days', 'is_new_customer',
       'total_order_value', 'pct_discount', 'days_to_delivery',
       'delivery_delay_days', 'is_late_delivery', 'order_year', 'order_month',
       'order_month_year', 'order_week', 'holiday_tag', 'basket_size_flag',
       'high_ticket_flag', 'customer_lifetime_orders', 'customer_lifetime_gmv',
       'customer_avg_ticket_to_date', 'prev_order_date',
       'days_since_last_order_before_purchase', 'region',
       'is_new_customer_period'],
      dtype='object')

In [ ]:
# Volume e valor

tmp = (
    orders.groupby("customer_id")
    .agg(
        total_orders=("order_id","count"),
        total_gmv=("order_value","sum"),
        avg_order_value=("order_value","mean"),
        median_order_value=("order_value","median"),
        total_discount=("discount_value","sum"),
        avg_pct_discount=("pct_discount","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mix e perfil de compra

tmp = (
    orders.groupby("customer_id")
    .agg(
        unique_categories=("order_category","nunique"),
        avg_sku_per_order=("sku_count","mean"),
        pct_high_ticket=("high_ticket_flag","mean"),  # proporção Q4
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Logística e experiência

tmp = (
    orders.groupby("customer_id")
    .agg(
        avg_days_to_delivery=("days_to_delivery","mean"),
        pct_deliveries_late=("is_late_delivery","mean"),
    )
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Região mais frequente

tmp = (
    orders.groupby("customer_id")["region"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_frequent_region")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Recompra e relacionamento

tmp = (
    orders.groupby("customer_id")["days_since_last_order_before_purchase"]
    .agg(avg_days_between_orders="mean", max_days_between_orders="max")
    .reset_index()
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Método de pagamento mais usado

tmp = (
    orders.groupby("customer_id")["payment_method"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name="most_used_payment")
)
customers = customers.merge(tmp, on="customer_id", how="left")

In [ ]:
# Mais algumas temporais

latest_date = orders["order_date"].max()
customers["days_since_last_order"] = (today - customers["last_order_date"]).dt.days
customers["is_active_90d"] = (customers["days_since_last_order"] <= 90).astype("Int8")

In [ ]:
# Coorte de entrada
customers["first_order_date"] = pd.to_datetime(customers["first_order_date"], errors="coerce")
customers["cohort_month"] = customers["first_order_date"].dt.to_period("M").astype(str)

In [ ]:
customers.columns

Index(['customer_id', 'first_order_date', 'last_order_date',
       'total_days_as_customer', 'total_orders', 'total_gmv',
       'avg_order_value', 'median_order_value', 'total_discount',
       'avg_pct_discount', 'unique_categories', 'avg_sku_per_order',
       'pct_high_ticket', 'avg_days_to_delivery', 'pct_deliveries_late',
       'most_frequent_region', 'avg_days_between_orders',
       'max_days_between_orders', 'most_used_payment', 'days_since_last_order',
       'is_active_90d', 'cohort_month'],
      dtype='object')

In [ ]:
customers.to_excel("customers_features.xlsx")